# Model - XGRBoost

In [4]:
import sys
import os
import yfinance as yf

root_path = os.path.abspath("..")   # go up one directory
sys.path.append(root_path)

from Project.param import *
from Project.data import prepare_stock_data, load_and_prepare_data

In [5]:
df = yf.download(TICKER, start=START_DATE, end=END_DATE)
df.head()

/var/folders/8y/wwgq01wx345gslv488wqly7m0000gn/T/ipykernel_18257/138827824.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(TICKER, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.418383,6.433078,6.369497,6.400988,493729600
2010-01-05,6.429478,6.465767,6.395588,6.436076,601904800
2010-01-06,6.327210,6.454972,6.320612,6.429480,552160000
2010-01-07,6.315515,6.358103,6.269629,6.350605,477131200
2010-01-08,6.357500,6.358100,6.269927,6.307115,447610800


In [6]:
drop_columns = ['Volume_PPH', 'Open_PPH', 'Low_PPH', 'High_PPH']

X_train, y_train, X_valid, y_valid, X_test, y_test= prepare_stock_data(
    ticker=TICKER,
    start=START_DATE,
    end=END_DATE,
    target_col=TARGET,
    drop_cols=drop_columns
)

/Users/hippolyteheger/code/ArnaudThs/StockProphet/StockProphet/Project/data.py:33: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed


In [7]:
X_train

,High_AAPL,Low_AAPL,Open_AAPL,Volume_AAPL,EMA_9,SMA_5,SMA_10,SMA_15,SMA_30,MACD,MACD_signal,RSI
0,7.205048,7.107877,7.143867,447017200.0,6.851751,7.085565,6.984765,6.896281,6.651234,0.200329,0.188490,78.179603
1,7.255434,7.157663,7.184355,628502000.0,6.885010,7.107699,7.018295,6.926452,6.693701,0.203078,0.191408,78.594248
2,7.244036,7.139068,7.211046,572989200.0,6.918119,7.141289,7.051975,6.959402,6.733629,0.201368,0.193400,83.855937
3,7.254533,7.211645,7.240737,334182800.0,6.945962,7.165161,7.091863,6.989993,6.771568,0.202114,0.195143,83.437696
4,7.289925,7.252136,7.263832,333026400.0,6.976538,7.200070,7.124523,7.029061,6.808727,0.201586,0.196432,81.135243
...,...,...,...,...,...,...,...,...,...,...,...,...
1943,37.562826,36.869767,36.979443,196458400.0,37.395289,37.286072,37.622798,37.707662,37.371564,0.207829,0.376912,44.933095
1944,37.453144,36.869762,37.362137,113077600.0,37.386607,37.344408,37.525490,37.707972,37.403469,0.159252,0.333380,36.286965
1945,37.282801,36.972441,37.222127,83242400.0,37.350557,37.212330,37.399946,37.664413,37.407211,0.120495,0.290803,34.681699
1946,36.930447,35.896695,36.846440,211805600.0,37.319513,37.112919,37.321773,37.599385,37.401581,0.039243,0.240491,26.799174


In [8]:
%%time
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
model = xgb.XGBRegressor(n_estimators=  400,
    learning_rate = 0.05,
    max_depth = 8,
    gamma = 0.005,
    random_state = 42, objective='reg:squarederror')
eval_set = [(X_train, y_train), (X_valid, y_valid)]

model.fit(X_train, y_train, eval_set=eval_set, verbose=False)
''''
parameters = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_depth': [8, 10, 12, 15],
    'gamma': [0.001, 0.005, 0.01, 0.02],
    'random_state': [42]
}

eval_set = [(X_train, y_train), (X_valid, y_valid)]
model = xgb.XGBRegressor(eval_set=eval_set, objective='reg:squarederror', verbose=False)
clf = GridSearchCV(model, parameters)

clf.fit(X_train, y_train)

print(f'Best params: {clf.best_params_}')
print(f'Best validation score = {clf.best_score_}')
'''

CPU times: user 578 ms, sys: 259 ms, total: 837 ms
Wall time: 541 ms


"'\nparameters = {\n    'n_estimators': [100, 200, 300, 400],\n    'learning_rate': [0.001, 0.005, 0.01, 0.05],\n    'max_depth': [8, 10, 12, 15],\n    'gamma': [0.001, 0.005, 0.01, 0.02],\n    'random_state': [42]\n}\n\neval_set = [(X_train, y_train), (X_valid, y_valid)]\nmodel = xgb.XGBRegressor(eval_set=eval_set, objective='reg:squarederror', verbose=False)\nclf = GridSearchCV(model, parameters)\n\nclf.fit(X_train, y_train)\n\nprint(f'Best params: {clf.best_params_}')\nprint(f'Best validation score = {clf.best_score_}')\n"

In [10]:
'''
model = xgb.XGBRegressor(**clf.best_params_, objective='reg:squarederror')
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)
'''

"\nmodel = xgb.XGBRegressor(**clf.best_params_, objective='reg:squarederror')\nmodel.fit(X_train, y_train, eval_set=eval_set, verbose=False)\n"

In [ ]:
import numpy as np

y_pred = model.predict(X_test)
print(f'y_true = {np.array(y_test)[:5]}')
print(f'y_pred = {y_pred[:5]}')


y_true = [50.25686646 49.92980576 50.55050659 49.76987076 48.42822266]
y_pred = [36.421547 36.1046   36.537426 36.58316  36.53356 ]


In [ ]:
from sklearn.metrics import mean_squared_error

print(f'mean_squared_error = {mean_squared_error(y_test, y_pred)}')


mean_squared_error = 2021.2133777910128


In [13]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Prepare predicted_prices DataFrame
predicted_prices = X_test.copy()
predicted_prices['Predicted'] = y_pred
predicted_prices['Actual'] = y_test.values

# Use the index as Date
predicted_prices = predicted_prices.reset_index()  # moves index to a column called 'index'
predicted_prices.rename(columns={'index': 'Date'}, inplace=True)

# Create figure
fig = make_subplots(rows=1, cols=1)

# True prices
fig.add_trace(go.Scatter(
    x=predicted_prices['Date'],
    y=predicted_prices['Actual'],
    name='Actual',
    mode='lines',
    line=dict(color='LightSkyBlue')
))

# Predicted prices
fig.add_trace(go.Scatter(
    x=predicted_prices['Date'],
    y=predicted_prices['Predicted'],
    name='Predicted',
    mode='lines',
    line=dict(color='MediumPurple')
))

# Layout
fig.update_layout(
    title="Stock Price: Actual vs Predicted",
    xaxis_title="Date",
    yaxis_title="Price",
    legend=dict(x=0, y=1),
    template="plotly_white",
    width=900,
    height=500
)

fig.show()
